# Importar Librerías Requeridas
Importa las librerías necesarias, incluyendo pandas para la manipulación de datos y geopy para la geocodificación.

In [1]:
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
import time
from datetime import datetime
import unicodedata
from multiprocessing import Pool, cpu_count
import os
from pyproj import Transformer

# Cargar Datos desde Archivo Parquet
Carga el DataFrame desde el archivo Parquet especificado que contiene la columna 'dirección'.

In [2]:
file_path = '../../data/processed/INEGI/AGEB_fm.csv'
df = pd.read_csv(file_path)
df.head()

/var/folders/c7/mdvpp7_n2dj49y88d6cj51kc0000gn/T/ipykernel_1144/3992418583.py:2: DtypeWarning: Columns (6,50,104,111,112,116,117) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


,ENTIDAD,NOM_ENT,MUN,NOM_MUN,LOC,NOM_LOC,AGEB,MZA,POBTOT,POBFEM,...,CVEREF2,TIPOVR2,NOMREF2,CVEVIAL3,CVESEG3,CVEREF3,TIPOVR3,NOMREF3,lon,lat
0,9,Ciudad de México,0,Total de la entidad Ciudad de México,0,Total de la entidad,0000,0,9209944,4805017,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,9,Ciudad de México,2,Azcapotzalco,0,Total del municipio,0000,0,432205,227255,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,9,Ciudad de México,2,Azcapotzalco,1,Total de la localidad urbana,0000,0,432205,227255,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,9,Ciudad de México,2,Azcapotzalco,1,Total AGEB urbana,0010,0,3183,1695,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,9,Ciudad de México,2,Azcapotzalco,1,Azcapotzalco,0010,1,159,86,...,2.0,Calle,Física,1618.0,3.0,1.0,Avenida,Civilizaciones,2.792381e+06,837760.8518


In [3]:
df = df[~df.lon.isnull()]

In [4]:
df[['lon', 'lat']] = df[['lon', 'lat']].astype(float)

In [5]:
# Limpia el nombre del municipio
df['municipio'] = df['NOM_MUN'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.lower().str.replace(' ', '_').str.replace('.', '')

In [6]:
# Selecciona solo un municipio para ejemplo
#df = df[df.municipio == 'benito_juarez'][:5]

In [7]:
df.columns[70:]

Index(['VPH_TINACO', 'VPH_CISTER', 'VPH_EXCSA', 'VPH_LETR', 'VPH_DRENAJ',
       'VPH_NODREN', 'VPH_C_SERV', 'VPH_NDEAED', 'VPH_DSADMA', 'VPH_NDACMM',
       'VPH_SNBIEN', 'VPH_REFRI', 'VPH_LAVAD', 'VPH_HMICRO', 'VPH_AUTOM',
       'VPH_MOTO', 'VPH_BICI', 'VPH_RADIO', 'VPH_TV', 'VPH_PC', 'VPH_TELEF',
       'VPH_CEL', 'VPH_INTER', 'VPH_STVP', 'VPH_SPMVPI', 'VPH_CVJ',
       'VPH_SINRTV', 'VPH_SINLTC', 'VPH_SINCINT', 'VPH_SINTIC', 'key',
       'ENTIDAD_fm', 'MUN_fm', 'LOC_fm', 'AGEB_fm', 'MZA_fm', 'CVEVIAL',
       'CVESEG', 'CVEFT', 'NOMVIAL', 'TIPOVIAL', 'CVEVIAL1', 'CVESEG1',
       'CVEREF1', 'TIPOVR1', 'NOMREF1', 'CVEVIAL2', 'CVESEG2', 'CVEREF2',
       'TIPOVR2', 'NOMREF2', 'CVEVIAL3', 'CVESEG3', 'CVEREF3', 'TIPOVR3',
       'NOMREF3', 'lon', 'lat', 'municipio'],
      dtype='object')

In [8]:
df[[ 'AGEB','MZA','NOMVIAL', 'TIPOVIAL', 'CVEVIAL1', 'CVESEG1',
       'CVEREF1', 'TIPOVR1', 'NOMREF1', 'CVEVIAL2', 'CVESEG2', 'CVEREF2',
       'TIPOVR2', 'NOMREF2', 'CVEVIAL3', 'CVESEG3', 'CVEREF3', 'TIPOVR3',
       'NOMREF3']].head()

,AGEB,MZA,NOMVIAL,TIPOVIAL,CVEVIAL1,CVESEG1,CVEREF1,TIPOVR1,NOMREF1,CVEVIAL2,CVESEG2,CVEREF2,TIPOVR2,NOMREF2,CVEVIAL3,CVESEG3,CVEREF3,TIPOVR3,NOMREF3
4,0010,1,Matemáticas,Calle,1615.0,1.0,4.0,Calle,Química,1613.0,1.0,2.0,Calle,Física,1618.0,3.0,1.0,Avenida,Civilizaciones
5,0010,1,Civilizaciones,Avenida,1615.0,1.0,4.0,Calle,Química,1613.0,1.0,2.0,Calle,Física,1623.0,3.0,3.0,Calle,Matemáticas
6,0010,1,Física,Calle,1623.0,3.0,3.0,Calle,Matemáticas,1618.0,3.0,1.0,Avenida,Civilizaciones,1615.0,1.0,4.0,Calle,Química
7,0010,1,Química,Calle,1623.0,3.0,3.0,Calle,Matemáticas,1618.0,3.0,1.0,Avenida,Civilizaciones,1613.0,1.0,2.0,Calle,Física
8,0010,2,Civilizaciones,Avenida,1613.0,1.0,4.0,Calle,Física,908.0,1.0,2.0,Calle,Cultura Náhuatl,1623.0,2.0,3.0,Calle,Matemáticas


In [9]:
# Función para convertir UTM (Easting, Northing) a lat/lon (WGS84)
def inegi_lcc_to_latlon(easting, northing):
    '''Convierte coordenadas Lambert Conformal Conic INEGI (EPSG:6372) a lat/lon WGS84.'''
    transformer = Transformer.from_crs("epsg:6372", "epsg:4326", always_xy=True)
    lon, lat = transformer.transform(easting, northing)
    return lat, lon

In [10]:
inegi_lcc_to_latlon(2.797545e+06, 825591.332517) 

(19.401674772489816, -99.15885625678762)

In [11]:
# Función para realizar la geocodificación inversa
def reverse_geocode(row):
    geolocator = Nominatim(user_agent="geocoder_app")
    try:
        lat, lon = inegi_lcc_to_latlon(row['lon'], row['lat'])  # lon=easting, lat=northing
        location = geolocator.reverse(f"{lat}, {lon}", timeout=5)
        if location:
            return location.raw['address']
        else:
            return None
    except GeocoderTimedOut:
        print("Timeout occurred")
        return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

In [12]:
def process_municipality(mun_name, mun_df):
    print(f"Processing municipality: {mun_name}")
    mun_df['address'] = mun_df.apply(reverse_geocode, axis=1)
    mun_df['road'] = mun_df['address'].apply(lambda x: x.get('road') if x else None)
    mun_df['quarter'] = mun_df['address'].apply(lambda x: x.get('quarter') if x else None)
    mun_df['borough'] = mun_df['address'].apply(lambda x: x.get('borough') if x else None)
    mun_df['postcode'] = mun_df['address'].apply(lambda x: x.get('postcode') if x else None)
    now = datetime.now()
    timestamp = now.strftime("%Y%m%d%H%M%S")
    filename = f"{timestamp}_{mun_name}.csv"
    filepath = os.path.join('../../data/processed/INEGI/colonia', filename)
    mun_df.to_csv(filepath, index=False)
    print(f"Saved municipality {mun_name} to {filepath}")
    return mun_df

In [13]:
df.municipio.value_counts()

municipio
iztapalapa                77476
gustavo_a_madero          40933
tlalpan                   35300
xochimilco                31108
alvaro_obregon            30300
coyoacan                  24760
tlahuac                   21448
azcapotzalco              15795
venustiano_carranza       15155
iztacalco                 13975
miguel_hidalgo            12421
cuauhtemoc                12044
la_magdalena_contreras    11057
milpa_alta                10769
cuajimalpa_de_morelos     10594
benito_juarez             10015
Name: count, dtype: int64

In [14]:
orden = df.municipio.value_counts(ascending=True).index
orden

Index(['benito_juarez', 'cuajimalpa_de_morelos', 'milpa_alta',
       'la_magdalena_contreras', 'cuauhtemoc', 'miguel_hidalgo', 'iztacalco',
       'venustiano_carranza', 'azcapotzalco', 'tlahuac', 'coyoacan',
       'alvaro_obregon', 'xochimilco', 'tlalpan', 'gustavo_a_madero',
       'iztapalapa'],
      dtype='object', name='municipio')

In [15]:
orden = ['tlalpan', 'gustavo_a_madero', 'iztapalapa']

In [16]:
# Ejemplo de procesamiento para el municipio seleccionado
for municipio in orden:
    mun_df = df[df.municipio == municipio].copy()
    process_df = process_municipality(municipio, mun_df)
    del process_df

Processing municipality: xochimilco
Saved municipality xochimilco to ../../data/processed/INEGI/colonia/20250929081834_xochimilco.csv
Processing municipality: tlalpan


KeyboardInterrupt: 

In [ ]:
process_df.address.values[4]